In [1]:
import pandas as pd

In [10]:
df = pd.read_csv("../sample_data/data.csv", sep=";", index_col=0)
df["Date"] = pd.to_datetime(df["Date"])
df

,Note,Amount,Category,Date
ID,,,,
1.0,Apartment,-500.0,Rent,2017-01-01
2.0,Apartment,-500.0,Rent,2017-01-02
3.0,Apartment,-500.0,Rent,2017-01-03
4.0,Apartment,-500.0,Rent,2017-01-04
5.0,Apartment,-500.0,Rent,2017-01-05
6.0,Apartment,-500.0,Rent,2017-01-06
7.0,Apartment,-500.0,Rent,2017-01-07
8.0,Apartment,-500.0,Rent,2017-01-08
9.0,Apartment,-500.0,Rent,2017-01-09


In [6]:
df.Event.unique()

array(['No'], dtype=object)